In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('diabetes.csv')
dataset.shape

(768, 9)

In [3]:
dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
zero_not_accepted = ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI', 'Insulin']
for c in zero_not_accepted:
    ##check for zero
    zero_before = dataset[c].where(dataset[c] == 0).count()
    if zero_before > 0:
        dataset[c] = dataset[c].replace(0, np.nan)
        mean_c = dataset[c].mean(skipna=True)
        dataset[c] = dataset[c].replace(np.nan, mean_c)
    ##check for zero
    zero_after = dataset[c].where(dataset[c] == 0).count()
    print(f"Col:{c} | zero_before: {zero_before}; zero_after: {zero_after} ")

Col:Glucose | zero_before: 5; zero_after: 0 
Col:BloodPressure | zero_before: 35; zero_after: 0 
Col:SkinThickness | zero_before: 227; zero_after: 0 
Col:BMI | zero_before: 11; zero_after: 0 
Col:Insulin | zero_before: 374; zero_after: 0 


In [5]:
from sklearn.model_selection import train_test_split
X = dataset.drop(['Outcome'], axis=1)
y = dataset['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit_transform(X_train)
sc.fit_transform(X_test)

array([[ 0.58741335,  1.33540403, -0.44986626, ...,  0.21147953,
         0.41908802,  1.21210448],
       [-0.86229268, -0.4119874 ,  0.96106164, ..., -0.2965198 ,
        -0.93981532, -0.85487437],
       [ 2.61700179, -0.63850111, -0.13632673, ..., -0.13686287,
        -0.04316189,  0.33761343],
       ...,
       [ 0.29747215, -0.4443465 , -0.44986626, ..., -0.89160473,
        -0.52490426, -0.29838006],
       [-0.28241027,  1.69135413,  0.96106164, ...,  0.16793673,
         1.87545368,  1.45060204],
       [-0.28241027, -0.3796283 ,  0.02044304, ..., -0.0788058 ,
        -0.78944487, -0.695876  ]])

In [7]:
from math import sqrt,floor
k_expected = y_test.shape[0]
k_expected = sqrt(k_expected)
k_expected-= 1
k_expected = floor(k_expected)
k = k_expected
k

11

In [8]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=k, p=2, metric='euclidean')
knn.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=11)

In [9]:
pred_df = X_test.copy()
pred_df
pred_df['actual'] = y_test
pred_df['predicted'] = knn.predict(X_test)
pred_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,actual,predicted
670,6,165.0,68.0,26.00000,168.000000,33.6,0.631,49,0,1
249,1,111.0,86.0,19.00000,155.548223,30.1,0.143,23,0,0
635,13,104.0,72.0,29.15342,155.548223,31.2,0.465,38,1,1
224,1,100.0,66.0,15.00000,56.000000,23.6,0.666,26,0,0
549,4,189.0,110.0,31.00000,155.548223,28.5,0.680,37,0,1


In [10]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(pred_df['actual'], pred_df['predicted'])
cm

array([[79, 20],
       [27, 28]])

In [13]:
from sklearn.metrics import classification_report
print(classification_report(pred_df['actual'], pred_df['predicted']))

              precision    recall  f1-score   support

           0       0.75      0.80      0.77        99
           1       0.58      0.51      0.54        55

    accuracy                           0.69       154
   macro avg       0.66      0.65      0.66       154
weighted avg       0.69      0.69      0.69       154

